In [ ]:
import numpy as np
import tensorflow as tf
from node.base import get_node_function
from node.fix_grid import RKSolver


# for reproducibility
np.random.seed(42)
tf.random.set_seed(42)


@tf.function
def dsigmoid(x):
    return tf.nn.sigmoid(x) * (1 - tf.nn.sigmoid(x))


@tf.function
def inv_sigmoid(x):
    return tf.math.log(x + 1e-8) - tf.math.log(1 - x + 1e-8)


@tf.function
def softmax(x, axis):
    return tf.nn.log_softmax(x, axis)


@tf.function
def softmin(x, axis):
    return -tf.nn.log_softmax(-x, axis)


@tf.function
def softrescale(x, axis):
    max = softmax(x, axis)
    min = softmin(x, axis)
    return (x - min) / (max - min)


@tf.function
def rescale(x, axis):
    max = tf.reduce_max(x, axis, keepdims=True)
    min = tf.reduce_min(x, axis, keepdims=True)
    return (x - min) / (max - min)


input_dim = 28 * 28
network = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu'),
    # tanh output for bounding the scale of phase vector field
    tf.keras.layers.Dense(input_dim, activation='tanh'),
])
network.build([None, input_dim])


class MyLayer(tf.keras.layers.Layer):

    def __init__(self, network, dt, num_grids, **kwargs):
        super().__init__(**kwargs)
        self.network = network
        self.dt = dt
        self.num_grids = num_grids

        t0 = tf.constant(0.)
        self.tN = t0 + num_grids * dt

#         def pvf(t, x):
#             r"""
#             $x^{\prime} = \sigma\left(\sigma^{-1}(x) + \Delta t f(t, x; \theta) \right)$,
#             element-wisely.
#             """
#             return dsigmoid(inv_sigmoid(x)) * self.network(x)

        def pvf(t, x):
            with tf.GradientTape() as g:
                g.watch(x)
                f = self.network(x)
                r = rescale(x, axis=-1)
            return g.gradient(r, [x], [f])[0]

        self._pvf = pvf
        self._node_fn = get_node_function(RKSolver(self.dt), 0., pvf)

    def call(self, x):
        y = self._node_fn(self.tN, x)
        return y


def process(X, y):
    X = X / 255.
    X = tf.reshape(X, [-1, 28 * 28])
    y = tf.one_hot(y, 10)
    return tf.cast(X, tf.float32), tf.cast(y, tf.float32)


mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, y_train = process(x_train, y_train)
x_test, y_test = process(x_test, y_test)

my_layer = MyLayer(network, dt=1e-2, num_grids=100)
output_layer = tf.keras.layers.Dense(
    10, activation='softmax',
    kernel_regularizer=tf.keras.regularizers.l2(1.))
model = tf.keras.Sequential([my_layer, output_layer])
model.build([None, 28 * 28])

optimizer = tf.compat.v1.train.AdamOptimizer()
loss_fn = tf.losses.CategoricalCrossentropy()


@tf.function
def train_one_step(x, y):
    with tf.GradientTape() as tape:
        outputs = model(x)
        loss = loss_fn(y, outputs)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss


@tf.function
def train(dataset):
    step, loss, reg = 0, float('inf'), float('inf')
    for x, y in dataset:
        loss = train_one_step(x, y)
        if step % 10 == 0:
            tf.print(step, loss)
        step += 1
    return loss


def clip(min, max, x):
    min = np.ones_like(x) * min
    max = np.ones_like(x) * max
    x = np.where(x < min, min, x)
    x = np.where(x > max, max, x)
    return x


num_epochs = 3
batch_size = 128

dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
dataset = dataset.repeat(num_epochs).batch(batch_size)

train(dataset)

0 2.44640827
10 1.79321373
20 1.5027082
30 1.51360345
40 1.46247411
50 1.24658418
60 1.00590777
70 0.648667455
80 0.665576041
90 0.944899738
100 0.636870742
110 0.717926
120 0.528918326
130 0.545190871
140 0.484120131
150 0.51841867
160 0.436444908
170 0.257186174
180 0.501385272
190 0.473104894
200 0.386331707
210 0.439368129
220 0.415830791
230 0.371607542
240 0.452593982
250 0.478032589
260 0.494715452
270 0.461615682
280 0.355713934
290 0.401416779
300 0.3571136
310 0.460269451
320 0.431369036
330 0.563382506
340 0.270792365
350 0.540035188
360 0.465415478
370 0.370710969
380 0.337132603
390 0.364546686
400 0.519720078
410 0.226486772
420 0.277364701
430 0.29386735
440 0.31721893
450 0.417874634
460 0.130583793
470 0.475861669
480 0.283348173
490 0.394779623
500 0.310987741
510 0.399646819
520 0.299859494
530 0.485628366
540 0.341901809
550 0.242458194
560 0.594061136
570 0.594908237
580 0.503781199
590 0.264123291
600 0.346892059
610 0.251698017
620 0.284000784
630 0.428739101


In [ ]:
from node.utils import tracer

def flip(array, ratio):
    is_flipped = np.random.random(size=array.shape) < ratio
    return np.where(is_flipped, 1 - array, array)

t0 = 0.
t1 = 1.
dt = 1e-2
traj_size = int((t1 - t0) / dt) + 1

n_data = 20
flip_ratio = 0.1

involved_labels = {1, 3, 5}

i, data, labels = 0, [], []
for x, y in zip(x_train, y_train):
    y = np.argmax(y)
    if y not in involved_labels:
        continue
    data.append(x)
    labels.append(y)
    data.append(flip(x, flip_ratio))
    labels.append(y)
    i += 1
    if i == n_data:
        break
data = np.array(data)
labels = np.array(labels)

trace = tracer(RKSolver(1e-2), my_layer._pvf)
trajectories = trace(t0, t1, dt, data)
trajectories = tf.transpose(trajectories, [1, 0, 2])
trajectories = trajectories.numpy()


def get_trajectory(x):
    """Input shape `[28 * 28]`, output shape `[frames, 28, 28]`."""
    trajectory = trace(t0, t1, dt, [x]).numpy()[:,0,:]
    trajectory = np.reshape(trajectory, [28, 28])
    return trajectory

In [ ]:
preds = output_layer(trajectories[:,-1,:]).numpy()

In [ ]:
i = 10
print(preds[i])
labels[i], np.argmax(preds[i])

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import animation


def rescale(array):
    shape = array.shape
    y = np.reshape(array, [-1])
    y = (y - np.min(y)) / (np.max(y) - np.min(y))
    return np.reshape(y, shape)


def visualize_trajectory(trajectory):
    """
    Args:
        trajectory: np.array
            Shape `[frames, x_pixal, y_pixal]`.
    
    Returns: animation.FuncAnimation
    """
    fig = plt.figure()
    ax = plt.axes()
    img = ax.imshow(trajectory[0], cmap='gray')

    def init():
        img.set_data([[]])
        return img,

    def animate(i):
        y = rescale(trajectory[i])
        img.set_data(y)
        return img,

    anim = animation.FuncAnimation(
        fig, animate, init_func=init, frames=traj_size, blit=True)
    return anim


for i, trajectory in enumerate(trajectories):
    label = labels[i]
    anim = visualize_trajectory(trajectory.reshape([-1, 28, 28]))
    anim.save(f'../dat/trajectory/anim_i{i}_l{label}.mp4')
    plt.show()

In [ ]:
labels

## Conclusion

### Approach 1

~~1. Animation plotting shows that attractors exist for all the displayed instances.~~

~~1. By tracing the flipping ratio, we find that, while setting $\tilde{L} = 3$ in the training process, the flip ratio decreases from $\sim 0.1$ to $\sim 0.001$ only after $\tilde{L} > 30$ approximately for all trials. That is, the static phase vector field is trained without reaching the attractors. And when reaching the attractors, instances in the same class have little difference (but not vanishing), instances from different classes become evidently more distinct.~~

~~1. The attractors for the same class, even though close to each other, are far from single. It seems to confirm the conclusion in the study of Hebbian learning that high-dimensional dynamic systems have extremely many attractors.~~

1. After re-scaling by `lambda x: (x - min(x)) / (max(x) - min(x))`, there does exists attractors having the properties described above. So, it seems that the phase point flying straightly towords some direction specific for different classes. Or say, "attracted to the direction".

1. It seems that we encountered the chaos along the phase trajectory.

### Approach 2

$x^{\prime} = \sigma\left(\sigma^{-1}(x) + \Delta t f(t, x; \theta) \right)$

1. Attractors are reached at $L \sim 300$.
1. There are quite a lot of attractors.
1. Indeed, in this case, the $x^{\alpha} = 0, 1$ for $\forall \alpha$ are attractors.
1. However, this approach introduces an artificial area of fixed points (i.e. $x^{\alpha} = 0, 1$ for $\forall \alpha$), which is not what we hope for.

## References:

1. [Chaos appears in RGE (as a high dimensional non-linear ODE)](https://physics.stackexchange.com/a/55057) (the [paper](https://arxiv.org/abs/hep-th/0304178) related).